In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model 
import os, shutil,pathlib,exifread
import pandas as pd
import numpy as np


# Define a function that utilistes our colour image classificaiton model, input is the file name to be predicted.
def sp_predict_colour(image_name):
    
    # Load the model weights
    new_model = load_model("Species_Classification_Model.h5")
    
    # Define the image size
    image_size = (200,200)
    
    # define the file path for the image to be predicted image_name is defined by the input from the function
    test_image = os.path.abspath(image_dir/image_name)
                                 
    # Preprocess the image to the same format as the images used when training the image
    img = keras.preprocessing.image.load_img(
    test_image,target_size = image_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array,0)
                                 
    # Store the predicted output                             
    pred = new_model.predict(img_array) 
    list = pred.flatten()
                                 
    # Multiply the scores to change them to a percentage                                 
    probability = list*100
    comp = {spp_list[i]:probability[i] for i in range(len(spp_list))}
    dic = comp
    
    # Store all of the scores into a dataframe
    df = pd.DataFrame(probability,spp_list,columns=["Species Match (%)"])
    species = df.sort_values(by=["Species Match (%)"], ascending=False).round(2)
    
    # Return the dataframe as the output of the defined function 
    return species

# Define a function that utilises the black and white image classification model, input is the file name to be predicted.
def sp_predict_bw(image_name):
    
    # Load the model weights
    new_model = load_model("Species_Classification_Model_Gray.h5")
    
    # Define the image size + ,1 on the end to specify the inputted image will not be RGB scale
    image_size = (200,200,1)
    
    # Define the image file path (image_name) from the input of the function 
    test_image = os.path.abspath(image_dir/image_name)
    
    # Load the image and apply the same preprocessing parameters that were used in the original model 
    img = keras.preprocessing.image.load_img(
    test_image,target_size = image_size)
    
    # Converts the image to greyscale if it wasn't already
    img = tf.image.rgb_to_grayscale(img)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array,0)
    
    # Store the predicted output
    pred = new_model.predict(img_array) 
    list = pred.flatten()
    
    # Multiply by 100 to show the percentage
    probability = list*100
    comp = {spp_list[i]:probability[i] for i in range(len(spp_list))}
    dic = comp
    
    # Store the output scores into a dataframe
    df = pd.DataFrame(probability,spp_list,columns=["Species Match (%)"])
    species = df.sort_values(by=["Species Match (%)"], ascending=False).round(2)
    
    # Return the scores dataframe as the output of the function. 
    return species

# Loop through the directory and use megadetector to detect animals and crop bounding boxes to save into a new directory. 

# Define the working directory path.
cwd = pathlib.Path("test_data")
root_dir = pathlib.Path("training_data/raw_images")

# Create an autonmous list that stores the names of all species in the training data
spp_list = []
for species in os.listdir(root_dir):
    spp_list.append(species)


# Create a dataframe that contains
my_data_df = pd.DataFrame()
df_columns = {"Date":(),"Location":(),"Feed":()}

for species in os.listdir(root_dir):
    df_columns.update({species:0})
for key in df_columns:
    my_data_df[key] = df_columns[key]

    
for location in os.listdir(cwd):
    #print(location,":")
    feed_type_dir = cwd/location
    for feed_type in os.listdir(feed_type_dir):
        date_dir = feed_type_dir/feed_type/"date"
        for date in os.listdir(date_dir):
            image_dir = date_dir/date/"cropped_images"
            sp_count_day = {"Date":date,"Location":location,"Feed":feed_type}
            for species in os.listdir(root_dir):
                sp_count_day.update({species:0})
                sp_count_day.update({"Unknown":0})
                sp_count_day.update({"Total_Count":0})
                
            for image in os.listdir(image_dir):
                print("Testing ",image," on Colour Model")
                species = sp_predict_colour(image)
                species_pred = species.index[0]
                species_pred_score = species["Species Match (%)"].iloc[0]
                print(species_pred,species_pred_score)
                if species_pred_score <= 50:
                    print("Testing on B/W Model")
                    species_bw = sp_predict_bw(image)
                    species_pred_bw = species_bw.index[0]
                    species_pred_score_bw = species_bw["Species Match (%)"].iloc[0]
                    print(species_pred_bw,species_pred_score_bw)
                    if species_pred_score_bw <=70:
                        print("Unknown Animal") 
                        sp_count_day["Unknown"]+=1
                        sp_count_day["Total_Count"]+=1
                    elif species_pred_score_bw > 70:
                        for species in spp_list:
                            if species_pred_bw == species:
                                sp_count_day[species] +=1
                                sp_count_day["Total_Count"]+=1
                elif species_pred_score >50:
                    for species in spp_list:
                        if species_pred == species:
                            sp_count_day[species] +=1
                            sp_count_day["Total_Count"]+=1
            sp_count_day_df = pd.DataFrame(sp_count_day,index=[0])
            my_data_df = pd.concat([my_data_df,sp_count_day_df],ignore_index=True)
            print("")
        print("")
    print("")
display(my_data_df)
my_data_df.to_csv("count_&_species_breakdown.csv",index =False)

Testing  11250245.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 147ms/step
Blue tit 99.08



Testing  12030071.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 140ms/step
Common Wood-pigeon 98.78
Testing  12030071.JPG___crop01_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 140ms/step
Great tit 98.84
Testing  12030284.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 138ms/step
Great tit 99.42
Testing  12030379.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 140ms/step
Great tit 99.32


Testing  12060105.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 139ms/step
Blue tit 99.99
Testing  12060109.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 146ms/step
Blue tit 95.89
Testing  12060109.JPG___crop01_md_v5a.0.0.pt

1/1 [==============================] - 0s 140ms/step
Eastern Gray Squirrel 54.22


Testing  12060213.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 140ms/step
Great tit 92.47
Testing  12060213.JPG___crop01_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 140ms/step
Nuthatch 36.91
Testing on B/W Model
1/1 [==============================] - 0s 134ms/step
Nuthatch 57.13
Unknown Animal
Testing  12060267.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 148ms/step
Eastern Gray Squirrel 33.59
Testing on B/W Model
1/1 [==============================] - 0s 142ms/step
Nuthatch 67.7
Unknown Animal





Testing  12210326.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 139ms/step
Great tit 56.79





,Date,Location,Feed,Blackbird,Blue tit,Coal tit,Common Wood-pigeon,Eastern Gray Squirrel,Eurasian bullfinch,Eurasian Jay,Eurasian wren,European robin,Great tit,Long-tailed tit,Nuthatch,Western Roe Deer,Unknown,Total_Count
0,2022_11_25,C2,fat_balls,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0
1,2022_11_26,C2,fat_balls,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
2,2022_11_27,C2,fat_balls,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
3,2022_12_03,C2,fat_balls,0,0,0,1,0,0,0,0,0,3,0,0,0,0.0,4.0
4,2022_12_05,C2,fat_balls,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
5,2022_12_06,C2,fat_balls,0,3,0,0,0,0,0,0,0,1,0,0,0,1.0,5.0
6,2022_12_07,C2,fat_balls,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
7,2022_12_08,C2,fat_balls,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
8,2022_12_09,C2,fat_balls,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,1.0
9,2022_12_12,C2,standard_seed,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,1.0
